### Dependencies

In [1]:
import pandas as pd

# Regex
import re

# NLTK
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

# TextBlob
from textblob import TextBlob

from langdetect import detect

In [ ]:
# Import

In [73]:
music_df = pd.read_csv("../api/data/lyrics_etl.csv")
music_df.head()

,Unnamed: 0,song_id,song_name,date_time_played,lyrics
0,0,3djNBlI7xOggg7pnsOLaNm,California Love - Original Version,2021-07-20T00:09:02.707Z,Chorus: Roger Troutman\nCalifornia love\nCalif...
1,1,1yNOoF6ctpmTL331UO4Alj,2 Of Amerikaz Most Wanted,2021-07-19T22:43:09.655Z,Intro: 2Pac & Snoop Doggy Dogg\nUp out of ther...
2,2,1yNOoF6ctpmTL331UO4Alj,2 Of Amerikaz Most Wanted,2021-07-19T21:19:11.977Z,Intro: 2Pac & Snoop Doggy Dogg\nUp out of ther...
3,3,6EZ9qFWLbRudnQdVA5hQRJ,Elephant,2021-07-19T16:08:32.085Z,Couldn't get lyrics for Elephant by Domastic.\n
4,4,56I4OIGTDhBmX8ZSJJDf1k,All I Need,2021-07-19T15:44:09.267Z,Intro\nMore time and more mind\n\nChorus\nGive...


### Regex Cleaning

In [75]:
def html_cleaner (string):
    html_cleaner = re.sub(r'\n', " ", string)
    return html_cleaner

In [76]:
music_df['lyrics'] = music_df.lyrics.apply(html_cleaner)
music_df.head()

,Unnamed: 0,song_id,song_name,date_time_played,lyrics
0,0,3djNBlI7xOggg7pnsOLaNm,California Love - Original Version,2021-07-20T00:09:02.707Z,Chorus: Roger Troutman California love Califor...
1,1,1yNOoF6ctpmTL331UO4Alj,2 Of Amerikaz Most Wanted,2021-07-19T22:43:09.655Z,Intro: 2Pac & Snoop Doggy Dogg Up out of there...
2,2,1yNOoF6ctpmTL331UO4Alj,2 Of Amerikaz Most Wanted,2021-07-19T21:19:11.977Z,Intro: 2Pac & Snoop Doggy Dogg Up out of there...
3,3,6EZ9qFWLbRudnQdVA5hQRJ,Elephant,2021-07-19T16:08:32.085Z,Couldn't get lyrics for Elephant by Domastic.
4,4,56I4OIGTDhBmX8ZSJJDf1k,All I Need,2021-07-19T15:44:09.267Z,Intro More time and more mind Chorus Give me ...


### Stop words

In [77]:
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to /Users/pierre-
[nltk_data]     olivierboisvert/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [78]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'under', "you're", 'won', 'ours', 'it', 'whom', 'm', "hasn't", 'you', 'to', 'o', 'needn', "weren't", 'were', 'few', "don't", 'themselves', 'yourselves', 'that', 'being', 'i', 'there', "needn't", 'then', 'again', 'because', 'his', 'yours', 'this', 'was', 'below', 'an', 'does', 'here', 'had', 'so', 'above', 'over', 'ain', "shouldn't", 'will', 'what', 'haven', 'but', 'with', 'for', 'isn', 'my', 'while', 'hadn', 'both', 'from', 'against', "wouldn't", 'been', "hadn't", 'into', 'she', 'shan', 'by', 'doing', 'them', 'more', "couldn't", 'most', 'down', 'ourselves', 'on', 'those', 'during', 'doesn', 'himself', 'any', 'him', 's', 'of', "didn't", 'which', 'until', "doesn't", 'are', 'is', 'has', "you've", 'other', 'who', 'not', 'own', 'after', 'why', "you'll", 'in', 'no', 'your', 've', "haven't", 'aren', "mustn't", 'yourself', 'just', 'do', 'weren', 'they', "it's", 'our', 'between', 'couldn', 'can', 'didn', 'd', 'wasn', 'myself', 'her', 'am', 'such', 'don', "should've", 'we', 're', 'these', 'must